# Step-by-step quick start guide to `CitationExtractor`

# Running the pipeline from the command-line

## Pre-processing

What does it do?:
- tokenization
- language identification
- sentence splitting (optional, depends on configuration)

In [10]:
!citedloci-pipeline --version

No handlers could be found for logger "citation_extractor.crfpp_wrap"
1.7.1


In [6]:
%%time
!citedloci-pipeline do preproc --config=config/project.ini

No handlers could be found for logger "citation_extractor.crfpp_wrap"
Tue, 22 Jun 2021 16:55:50 - citation_extractor.Utils.IO - [INFO] Logger initialised
Tue, 22 Jun 2021 16:55:50 - citation_extractor.Utils.IO - [INFO] Current working directory: /media/romanell/4T/matteo/ClassicsCitations/IndexLocorum101/data
Tue, 22 Jun 2021 16:55:50 - citation_extractor.Utils.IO - [INFO] There are 1 docs to process
Tue, 22 Jun 2021 16:55:50 - citation_extractor.Utils.IO - [INFO] Following documents will be processed: [u'bmcr_2013-01-10.txt']
Tue, 22 Jun 2021 16:55:50 - citation_extractor.Utils.IO - [INFO] Env variable $TREETAGGER_HOME == /home/romanell/tree-tagger/
Tue, 22 Jun 2021 16:55:50 - citation_extractor.Utils.IO - [INFO] Treetagger for fr successfully initialised
Tue, 22 Jun 2021 16:55:50 - citation_extractor.Utils.IO - [INFO] Treetagger for en successfully initialised
Tue, 22 Jun 2021 16:55:50 - citation_extractor.Utils.IO - [INFO] Treetagger for nl successfully initialised
Tue, 22 Jun 2021 

## Named entity recognition

Process all files:

In [3]:
!citedloci-pipeline do ner --config=config/project.ini

No handlers could be found for logger "citation_extractor.crfpp_wrap"
Tue, 22 Jun 2021 16:51:22 - citation_extractor.Utils.IO - [INFO] Logger initialised
Tue, 22 Jun 2021 16:51:22 - citation_extractor.Utils.IO - [INFO] Current working directory: /media/romanell/4T/matteo/ClassicsCitations/IndexLocorum101/data
Tue, 22 Jun 2021 16:51:22 - citation_extractor.Utils.IO - [INFO] There are 1 docs to process
Tue, 22 Jun 2021 16:51:22 - citation_extractor.Utils.IO - [INFO] Following documents will be processed: [u'bmcr_2013-01-10.txt']
Tue, 22 Jun 2021 16:51:22 - citation_extractor.Utils.IO - [INFO] Extractor loaded from pickle /media/romanell/4T/matteo/ClassicsCitations/IndexLocorum101/pickles/extractor.pkl
Tue, 22 Jun 2021 16:51:24 - citation_extractor.Utils.IO - [INFO] Output successfully written to file
Tue, 22 Jun 2021 16:51:24 - citation_extractor.io.converters - [WARNING] No knowledge based was provided, thus `self._convert_disambiguations()` won't work.
Tue, 22 Jun 2021 16:51:24 - citat

Process one specific document (from input folders):

In [7]:
!citedloci-pipeline do ner --config=config/project.ini --doc=bmcr_2013-01-10.txt

No handlers could be found for logger "citation_extractor.crfpp_wrap"
Tue, 22 Jun 2021 16:57:18 - citation_extractor.Utils.IO - [INFO] Logger initialised
Tue, 22 Jun 2021 16:57:18 - citation_extractor.Utils.IO - [INFO] Current working directory: /media/romanell/4T/matteo/ClassicsCitations/IndexLocorum101/data
Tue, 22 Jun 2021 16:57:18 - citation_extractor.Utils.IO - [INFO] There are 1 docs to process
Tue, 22 Jun 2021 16:57:18 - citation_extractor.Utils.IO - [INFO] Following documents will be processed: ['bmcr_2013-01-10.txt']
Tue, 22 Jun 2021 16:57:19 - citation_extractor.Utils.IO - [INFO] Extractor loaded from pickle /media/romanell/4T/matteo/ClassicsCitations/IndexLocorum101/pickles/extractor.pkl
Tue, 22 Jun 2021 16:57:20 - citation_extractor.Utils.IO - [INFO] Output successfully written to file
Tue, 22 Jun 2021 16:57:20 - citation_extractor.io.converters - [WARNING] No knowledge based was provided, thus `self._convert_disambiguations()` won't work.
Tue, 22 Jun 2021 16:57:20 - citati

In [8]:
cat data/json/bmcr_2013-01-10.json | jq ".entities"

{
  "11": {
    "end_offset": 8562,
    "entity_type": "AAUTHOR",
    "id": "11",
    "start_offset": 8553,
    "surface": "Thucydide"
  },
  "10": {
    "end_offset": 8427,
    "entity_type": "AAUTHOR",
    "id": "10",
    "start_offset": 8416,
    "surface": "Thucydides,"
  },
  "13": {
    "entity_type": "REFAUWORK",
    "urn": "urn:cts:greekLit:tlg0016.tlg001",
    "start_offset": 9859,
    "work_uri": "http://purl.org/hucit/kb/works/2691",
    "surface": "(Hdt.",
    "end_offset": 9864,
    "id": "13"
  },
  "12": {
    "end_offset": 9801,
    "entity_type": "AAUTHOR",
    "id": "12",
    "start_offset": 9792,
    "surface": "Herodotus"
  },
  "14": {
    "entity_type": "REFSCOPE",
    "norm_scope": "3.38.5",
    "start_offset": 9865,
    "surface": "3.38.5).",
    "end_offset": 9873,
    "id": "14"
  },
  "1": {
    "end_offset": 704,
    "entity_type": "AAUTHOR",
    "id": "1",
    "start_offset": 694,
    "surface": "Herodotus,"
  },
  "3": {
    "entity_type": "REFAUWORK",
   

## Relation extraction

In [9]:
!citedloci-pipeline do relex --config=config/project.ini

No handlers could be found for logger "citation_extractor.crfpp_wrap"
Tue, 22 Jun 2021 16:58:26 - citation_extractor.Utils.IO - [INFO] Logger initialised
Tue, 22 Jun 2021 16:58:26 - citation_extractor.Utils.IO - [INFO] Current working directory: /media/romanell/4T/matteo/ClassicsCitations/IndexLocorum101/data
Tue, 22 Jun 2021 16:58:26 - citation_extractor.Utils.IO - [INFO] There are 2 docs to process
Tue, 22 Jun 2021 16:58:26 - citation_extractor.Utils.IO - [INFO] Following documents will be processed: [u'bmcr_2013-01-10.json', u'capra2015_introduction.json']
Tue, 22 Jun 2021 16:58:26 - citation_extractor.Utils.IO - [INFO] Document bmcr_2013-01-10.json contains 14 entities.
Tue, 22 Jun 2021 16:58:26 - citation_extractor.Utils.IO - [INFO] Document bmcr_2013-01-10.json (14 entities, 4 relations) written to /media/romanell/4T/matteo/ClassicsCitations/IndexLocorum101/data/json/bmcr_2013-01-10.json
Tue, 22 Jun 2021 16:58:26 - citation_extractor.Utils.IO - [INFO] Document capra2015_introduct

In [18]:
cat data/json/bmcr_2013-01-10.json | jq ".|keys"

[
  "disambiguations",
  "doc_id",
  "doc_name",
  "entities",
  "linebreaks",
  "relations",
  "text",
  "tokens"
]


In [20]:
cat data/json/bmcr_2013-01-10.json | jq ".|.relations"

{
  "R4": [
    "13",
    "14"
  ],
  "R1": [
    "3",
    "4"
  ],
  "R2": [
    "5",
    "6"
  ],
  "R3": [
    "7",
    "8"
  ]
}


## Named entity linking

In [1]:
!citedloci-pipeline do ned --config=config/project.ini

No handlers could be found for logger "citation_extractor.crfpp_wrap"
Tue, 22 Jun 2021 21:33:13 - citation_extractor.Utils.IO - [INFO] Logger initialised
Tue, 22 Jun 2021 21:33:13 - citation_extractor.Utils.IO - [INFO] Current working directory: /media/romanell/4T/matteo/ClassicsCitations/IndexLocorum101/data
Tue, 22 Jun 2021 21:33:14 - citation_extractor.Utils.IO - [INFO] CitationMatcher loaded from pickle /media/romanell/4T/matteo/ClassicsCitations/IndexLocorum101/pickles/matcher.pkl
Tue, 22 Jun 2021 21:33:14 - citation_extractor.Utils.IO - [INFO] There are 1 docs to process
Tue, 22 Jun 2021 21:33:14 - citation_extractor.Utils.IO - [INFO] Following documents will be processed: [u'bmcr_2013-01-10.json']
(CITATION (REF (SCOPE_S (LEVEL 3) (LEVEL 38) (LEVEL 5))))
Tue, 22 Jun 2021 21:33:16 - citation_extractor.Utils.IO - [INFO] 3.38.5). => 3.38.5
(CITATION (REF (SCOPE_S (LEVEL 1) (LEVEL 101) (LEVEL 2))))
Tue, 22 Jun 2021 21:33:16 - citation_extractor.Utils.IO - [INFO] 1.101.2. => 1.101.2


In [2]:
cat data/json/bmcr_2013-01-10.json | jq ".|.entities"

{
  "11": {
    "end_offset": 8562,
    "entity_type": "AAUTHOR",
    "id": "11",
    "start_offset": 8553,
    "surface": "Thucydide"
  },
  "10": {
    "end_offset": 8427,
    "entity_type": "AAUTHOR",
    "id": "10",
    "start_offset": 8416,
    "surface": "Thucydides,"
  },
  "13": {
    "entity_type": "REFAUWORK",
    "urn": "urn:cts:greekLit:tlg0016.tlg001",
    "start_offset": 9859,
    "work_uri": "http://purl.org/hucit/kb/works/2691",
    "surface": "(Hdt.",
    "end_offset": 9864,
    "id": "13"
  },
  "12": {
    "end_offset": 9801,
    "entity_type": "AAUTHOR",
    "id": "12",
    "start_offset": 9792,
    "surface": "Herodotus"
  },
  "14": {
    "entity_type": "REFSCOPE",
    "norm_scope": "3.38.5",
    "start_offset": 9865,
    "surface": "3.38.5).",
    "end_offset": 9873,
    "id": "14"
  },
  "1": {
    "end_offset": 704,
    "entity_type": "AAUTHOR",
    "id": "1",
    "start_offset": 694,
    "surface": "Herodotus,"
  },
  "3": {
    "entity_type": "REFAUWORK",
   

# Read extracted citations

In [60]:
import pandas as pd

In [58]:
def read_json(doc_dir, doc_id):
    
    inp_file_path = os.path.join(doc_dir, doc_id)
    
    records = []
    
    # read input file
    with codecs.open(inp_file_path, 'r', 'utf-8') as inpfile:
        doc = json.load(inpfile)
        
    for relation in doc['relations']:
        arg1_id, arg2_id = doc['relations'][relation]
        arg1 = doc['entities'][arg1_id]
        arg2 = doc['entities'][arg2_id]
        
        if arg1['urn'] != 'urn:cts:GreekLatinLit:NIL':
            passage_urn = arg1['urn'] + ":" + arg2['norm_scope']
        else:
            passage_urn = None
        
        records.append({
            "docid": doc_id,
            "surface": arg1['surface'] + " " + arg2['surface'],
            "passage_urn": passage_urn,
            "work_urn": arg1['urn'],
            "work_uri": arg1['work_uri'] if 'work_uri' in arg1 else None
        })
        
    return record

In [59]:
data = read_json('data/json/', 'bmcr_2013-01-10.json')

In [61]:
df = pd.DataFrame(data).set_index('docid')

In [62]:
df

,passage_urn,surface,work_uri,work_urn
docid,,,,
bmcr_2013-01-10.json,urn:cts:greekLit:tlg0016.tlg001:3.38.5,(Hdt. 3.38.5).,http://purl.org/hucit/kb/works/2691,urn:cts:greekLit:tlg0016.tlg001
bmcr_2013-01-10.json,urn:cts:greekLit:tlg0003.tlg001:1.101.2,(Thuc. 1.101.2.,http://purl.org/hucit/kb/works/3998,urn:cts:greekLit:tlg0003.tlg001
bmcr_2013-01-10.json,urn:cts:greekLit:tlg0032.tlg001:3.3,Xen. Hell. 3.3),http://purl.org/hucit/kb/works/4025,urn:cts:greekLit:tlg0032.tlg001
bmcr_2013-01-10.json,None,"Dike 12-13,",None,urn:cts:GreekLatinLit:NIL
